# Data download

In [1]:
#stuff

In [6]:
import yfinance as yf
import pandas as pd

gold = yf.Ticker('GC=F')

#df = yf.download(gold, period="max")
df = gold.history(period='max')
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2000-08-30,273.899994,273.899994,273.899994,273.899994,0,0,0
2000-08-31,274.799988,278.299988,274.799988,278.299988,0,0,0
2000-09-01,277.000000,277.000000,277.000000,277.000000,0,0,0
2000-09-05,275.799988,275.799988,275.799988,275.799988,2,0,0
2000-09-06,274.200012,274.200012,274.200012,274.200012,0,0,0


In [7]:
df.shape

(5539, 7)